# Dynamics

Dynamics come in many flavors: stochastic vs. deterministic, discrete time vs. continuous time, and many different variations of the same model. For this reason, it is near-impossible to implement every possible dynamical model in software.

XGI not only incorporates basic dynamics, but also makes it easier to custom define your own dynamics. We illustrate this below.

In [ ]:
import random

import matplotlib.pyplot as plt
import numpy as np
import xgi

## Contagion dynamics

Here, we will build up contagion dynamics from the ground up for the hypergraph susceptible-infected-susceptible(SIS) model. This model comprises three different mechanisms:

1. Spontaneous healing: If a node is infected, then with probability $\gamma$, it recovers and becomes susceptible again.
2. Pairwise infection: If a node is susceptible, but its neighbor through a pairwise infection is infected, then it is infected with probability $\beta_2$.
2. Three-way infection: If a node is susceptible, but both of its neighbors in a three-way infection are infected, then it is infected with probability $\beta_3$.

In [ ]:
def SIS(H, beta, gamma, tmax):
    n = H.num_nodes
    num_seeds = 10
    # susceptible is 0, infected is 1
    X = np.zeros((tmax, n))
    X[0, random.sample(range(n), num_seeds)] = 1

    members = H.edges.members(dtype=dict)
    memberships = H.nodes.memberships()
    for t in range(tmax - 1):
        # loop over all the nodes
        X[t + 1] = X[t]
        for i in range(n):
            # infection
            if X[t, i] == 0:
                for e in memberships[i]:
                    # majority vote
                    edge = list(members[e])
                    if X[t, edge].sum() / len(edge) > 0.5 and random.random() < beta:
                        X[t + 1, i] = 1
                        break

            # healing
            if X[t, i] == 1 and random.random() < gamma:
                X[t + 1, i] = 0
    return np.arange(tmax), X


def SIR(H, beta, gamma, tmax):
    n = H.num_nodes
    num_seeds = 10
    # susceptible is 0, infected is 1
    X = np.zeros((tmax, n))
    X[0, random.sample(range(n), num_seeds)] = 1

    members = H.edges.members(dtype=dict)
    memberships = H.nodes.memberships()

    t = 0
    while t < tmax - 1 and sum(X[t] == 1) > 0:
        # loop over all the nodes
        X[t + 1] = X[t]
        for i in range(n):
            # infection
            if X[t, i] == 0:
                for e in memberships[i]:
                    # majority vote
                    edge = list(members[e])
                    if X[t, edge].sum() / len(edge) > 0.5 and random.random() < beta:
                        X[t + 1, i] = 1
                        break

            # healing
            if X[t, i] == 1 and random.random() < gamma:
                X[t + 1, i] = 2

        t += 1
    return np.arange(tmax), X

In [ ]:
n = 100
H = xgi.fast_random_hypergraph(n, [0.07, 0.05])

In [ ]:
t, X = SIR(H, 0.005, 0.1, 50)

In [ ]:
plt.spy(X.T)
plt.xlabel("Time")
plt.ylabel("Node state")

In [ ]:
plt.plot((X == 0).sum(axis=1), label="Susceptible")
plt.plot((X == 1).sum(axis=1), label="Infected")
plt.plot((X == 2).sum(axis=1), label="Recovered")

plt.xlabel("Time")
plt.xlabel("Number")
plt.legend()

In [ ]:
t, X = SIS(H, 0.005, 0.1, 50)

In [ ]:
plt.plot(X.sum(axis=1), label="Infected")

## Synchronization dynamics

Here, we will use the canned functionality in XGI for simulating the hypergraph Kuramoto model as described in [Adhikari *et al.* 2023](https://doi.org/10.1063/5.0116747).

In [ ]:
n = 100
H = xgi.fast_random_hypergraph(n, [0.05, 0.001], seed=None)
omega = 2 * np.random.normal(1, 0.05, n)
theta = np.linspace(0, 2 * np.pi, n)
timesteps = 500
dt = 0.01

theta, t = xgi.simulate_kuramoto(
    H, k2=2, k3=3, omega=omega, theta=theta, timesteps=timesteps, dt=dt
)
R = xgi.compute_kuramoto_order_parameter(theta)

In [ ]:
plt.plot(t, theta)
plt.show()

In [ ]:
plt.plot(t, R)

## Challenge:
* Using the `simulate_kuramoto` function with the hypergraph above, vary the value of `k2` while letting `k3=2`, and plot the steady-state order parameter (an approximation could be the value at t=5) with respect to `k2` to determine the critical value for sync to occur.